In [ ]:
# !pip install -r ../requirements.txt > ../.log

In [2]:
import asyncio
import os
import yt_dlp
import nest_asyncio
from moviepy import VideoFileClip
from pydub import AudioSegment
from shazamio import Shazam
from tabulate import tabulate
import ssl
import aiohttp

In [3]:
nest_asyncio.apply()

# Disable SSL verification
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

async def download_audio(youtube_url: str, output_path: str, cookies_path: str):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': output_path,
        'noplaylist': True,         # Ensure only the single video is downloaded
        'overwrites': True,         # Overwrite the file if it already exists
        'cookies': cookies_path,    # Use cookies file
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

async def recognize_song_from_youtube(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': 'temp_audio.%(ext)s',
        'quiet': True
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=False)
        audio_url = info_dict['url']
    shazam = Shazam()
    out = await shazam.recognize_song(audio_url)
    if 'track' in out:
        track = out['track']
        song_title = track.get('title', 'Unknown Title')
        artist_name = track.get('subtitle', 'Unknown Artist')
        album_name = track.get('sections', [{}])[0].get('metadata', [{}])[0].get('text', 'Unknown Album')
        genre = track.get('genres', {}).get('primary', 'Unknown Genre')
        print(f"Song: {song_title}, Artist: {artist_name}, Album: {album_name}, Genre: {genre}")
    else:
        print("Could not recognize the track.")

async def recognize_track(file_path: str, segment_duration: int = 8):
    shazam = Shazam()
    audio = AudioSegment.from_file(file_path)
    duration = len(audio) // 1000  # Convert to seconds
    results = []
    i = 0
    for start in range(0, duration, segment_duration):
        end = min(start + segment_duration, duration)
        segment = audio[start * 1000:end * 1000]
        file_path = f"../data/temp_segment{i}.mp3"
        segment.export(file_path, format="mp3")
        out = await shazam.recognize(file_path)
        os.remove(file_path)
        # print(out)
        if 'track' in out:
            track = out['track']
            song_title = track.get('title', 'Unknown Title')
            artist_name = track.get('subtitle', 'Unknown Artist')
            album_name = track.get('sections', [{}])[0].get('metadata', [{}])[0].get('text', 'Unknown Album')
            genre = track.get('genres', {}).get('primary', 'Unknown Genre')
            print(f"Segment {i}: {start}-{end}, Song: {song_title}, Artist: {artist_name}, Album: {album_name}, Genre: {genre}")
            results.append({
                'segment': i,
                'start': start,
                'end': end,
                'song': song_title,
                'artist': artist_name,
                'album': album_name,
                'genre': genre
            })
        else:
            print("Could not recognize the track.")
            results.append({
                'segment': i,
                'start': start,
                'end': end,
                'song': "None",
                'artist': None,
                'album': None,
                'genre': None
            })
        i += 1
    return results

def convert_mp4_to_mp3(mp4_file, mp3_file):
    video_clip = VideoFileClip(mp4_file)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(mp3_file)
    audio_clip.close()
    video_clip.close()

segment_duration = 6                                            # seconds
cookies_file = '../data/cookies.txt'                            # Path to your cookies file
# youtube_url = 'https://www.youtube.com/shorts/lTqjbKZZ9vk'      # hard
# youtube_url = 'https://www.youtube.com/shorts/IykG5LsziGw'      # easy
youtube_url = 'https://www.youtube.com/watch?v=7vglAmBc9tY'     # detroit lion
# audio_file = '../data/video.webm'
# await download_audio(youtube_url, audio_file, cookies_file)
# audio_file = '../data/harder.mp3'
# asyncio.run(recognize_song_from_youtube(youtube_url))

mp4_file = '../data/video_3_songs_mixed.mp4'
audio_file = '../data/video_audio.mp3'
convert_mp4_to_mp3(mp4_file, audio_file)

result = await recognize_track(audio_file, segment_duration)


{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '1', 'compatible_brands': 'isommp41mp42', 'creation_time': '2025-02-03T12:46:04.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 110, 'metadata': {'Metadata': '', 'creation_time': '2025-02-03T12:46:04.000000Z', 'handler_name': 'Core Media Audio', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 19548, 'fps': 59.94, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'creation_time': '2025-02-03T12:46:04.000000Z', 'handler_name': 'Core Media Video', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 146.07999999999998, 'bitrate': 19672, 'start': 0.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 0, 'audio_fps': 48000, 'a

MoviePy - Done.
Segment 0: 0-6, Song: Sugar, Artist: Maroon 5, Album: Will Be Loved EP, Genre: Pop
Segment 1: 6-12, Song: Sugar, Artist: Maroon 5, Album: Will Be Loved EP, Genre: Pop
Segment 2: 12-18, Song: What Makes You Beautiful, Artist: One Direction, Album: Up All Night, Genre: Pop
Segment 3: 18-24, Song: What Makes You Beautiful, Artist: One Direction, Album: Up All Night, Genre: Pop
Segment 4: 24-30, Song: What Makes You Beautiful, Artist: One Direction, Album: Up All Night, Genre: Pop
Segment 5: 30-36, Song: What Makes You Beautiful, Artist: One Direction, Album: Up All Night, Genre: Pop
Segment 6: 36-42, Song: What Makes You Beautiful, Artist: One Direction, Album: Up All Night, Genre: Pop
Segment 7: 42-48, Song: What Makes You Beautiful, Artist: One Direction, Album: Up All Night, Genre: Pop
Could not recognize the track.
Could not recognize the track.
Could not recognize the track.
Could not recognize the track.
Could not recognize the track.
Segment 13: 78-84, Song: Sugar, 

In [4]:
songs = []
cur = (result[0]['song'], result[0]['artist'])
start = result[0]['start']
end = result[0]['end']
for x in result:
    if x['song'] == cur[0] and x['artist'] == cur[1]:
        end = x['end']
    else:
        songs.append((cur[0], cur[1], start, end))
        cur = (x['song'], x['artist'])
        start = x['start']
        end = x['end']
songs.append((cur[0], cur[1], start, end))

headers = ['Song', 'Artist', 'Start (second)', 'End (second)']
print(tabulate(songs, headers=headers, tablefmt='plain'))

Song                          Artist           Start (second)    End (second)
Sugar                         Maroon 5                      0              12
What Makes You Beautiful      One Direction                12              48
None                                                       48              78
Sugar                         Maroon 5                     78              90
Last Friday Night (T.G.I.F.)  Katy Perry                   90             114
What Makes You Beautiful      One Direction               114             146


In [ ]:
# !pip freeze | sed 's/ @ file:.*//' > ../requirements.txt